# 04 - Modeling


## Objective
Stratified binning, baseline models, event-flag features, 5-fold CV stability, and artifact saving.

---

### Run order
Execute notebooks in numeric order: 01 → 02 → 03 → 04 → 05. Each notebook mounts Google Drive and reads/writes intermediate artifacts so it can run independently.

---


In [3]:

import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Ensure packages
!pip -q install lightgbm catboost

import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_hist_gradient_boosting  # noqa: F401
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from google.colab import drive

# Mount
if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')
else:
    print('Drive already mounted!')

PREP_IN = '/content/drive/MyDrive/dsp-poc/data/df_preprocessed.parquet'
ART_DIR = '/content/drive/MyDrive/dsp-poc/artifacts'
os.makedirs(ART_DIR, exist_ok=True)

df = pd.read_parquet(PREP_IN)
print('Loaded preprocessed:', df.shape)

# Features & bins
exclude_cols = ['RUL', 'unit', 'cycle', 'fail_in_H']
sensor_cols = [c for c in df.columns if c not in exclude_cols]

rul_max = float(df['RUL'].max())
bin_edges = [0, 30, 200, 500, rul_max + 1e-9]
df['rul_bin'] = pd.cut(df['RUL'], bins=bin_edges, include_lowest=True)

X = df[sensor_cols].copy()
y = df['RUL'].astype(float).copy()
bins = df['rul_bin'].astype(str)

# Stratified 60/40 split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.40, random_state=42)
for train_idx, test_idx in sss.split(X, bins):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    bins_train, bins_test = bins.iloc[train_idx], bins.iloc[test_idx]

# Internal val split
sss_val = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
for tr_idx, val_idx in sss_val.split(X_train, bins_train):
    X_tr, X_val = X_train.iloc[tr_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

# Utilities
np.random.seed(42)

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def evaluate_model(name, y_true, y_pred):
    return { 'Model': name, 'RMSE': rmse(y_true, y_pred), 'MAE': float(mean_absolute_error(y_true, y_pred)), 'R2': float(r2_score(y_true, y_pred)) }

metrics = []

# 1) RandomForest
rf = RandomForestRegressor(n_estimators=700, max_depth=16, min_samples_leaf=25, max_features='sqrt', n_jobs=-1, random_state=42)
rf.fit(X_tr, y_tr)
y_pred_rf = rf.predict(X_test)
metrics.append(evaluate_model('RandomForest', y_test, y_pred_rf))

# 2) HistGB
hgb = HistGradientBoostingRegressor(max_depth=6, learning_rate=0.06, max_iter=800, min_samples_leaf=20, l2_regularization=1.0, random_state=42)
hgb.fit(X_tr, y_tr)
y_pred_hgb = hgb.predict(X_test)
metrics.append(evaluate_model('HistGB', y_test, y_pred_hgb))

# 3) LightGBM baseline
lgbm = lgb.LGBMRegressor(objective='regression', learning_rate=0.05, num_leaves=31, max_depth=-1, min_child_samples=30, subsample=0.8, colsample_bytree=0.9, reg_lambda=1.0, n_estimators=5000, random_state=42)
lgbm.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], eval_metric='rmse', callbacks=[lgb.early_stopping(stopping_rounds=200, verbose=True), lgb.log_evaluation(period=200)])
best_iter = getattr(lgbm, 'best_iteration_', None)
y_pred_lgbm = lgbm.predict(X_test, num_iteration=best_iter) if best_iter else lgbm.predict(X_test)
metrics.append(evaluate_model('LightGBM', y_test, y_pred_lgbm))

# 4) CatBoost baseline
cat = CatBoostRegressor(loss_function='RMSE', eval_metric='RMSE', depth=6, learning_rate=0.05, l2_leaf_reg=3.0, bootstrap_type='Bernoulli', subsample=0.8, random_strength=0.5, iterations=5000, early_stopping_rounds=200, use_best_model=True, random_seed=42, verbose=200)
cat.fit(X_tr, y_tr, eval_set=(X_val, y_val))
y_pred_cat = cat.predict(X_test)
metrics.append(evaluate_model('CatBoost', y_test, y_pred_cat))

import pandas as pd
compare_df = pd.DataFrame(metrics).sort_values('RMSE')
print('=== Test Metrics (Baseline) ===')
print(compare_df)

# Event flags for sensor_04/05

def add_event_flags(df_in):
    df2 = df_in.copy()
    def robust_stats(s):
        q1, q3 = s.quantile(0.25), s.quantile(0.75)
        iqr = q3 - q1
        med = s.median()
        return med, q1, q3, iqr
    for s in ['sensor_04', 'sensor_05']:
        if s not in df2.columns:
            print(f'Warning: {s} not found; skipping flags.')
            continue
        med, q1, q3, iqr = robust_stats(df2[s])
        safe_iqr = iqr if iqr != 0 else 1.0
        df2[f'{s}_drop_flag']  = (df2[s] < (q1 - 1.5*safe_iqr)).astype(int)
        df2[f'{s}_spike_flag'] = (df2[s] > (q3 + 1.5*safe_iqr)).astype(int)
        df2[f'{s}_dev_med_iqr'] = (df2[s] - med) / safe_iqr
        unit_stats = df2.groupby('unit')[s].agg(['median', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)])
        unit_stats.columns = ['u_med', 'u_q1', 'u_q3']
        df2 = df2.join(unit_stats, on='unit')
        df2[f'{s}_unit_iqr'] = (df2['u_q3'] - df2['u_q1']).replace(0, np.nan).fillna(safe_iqr)
        df2[f'{s}_dev_unit'] = (df2[s] - df2['u_med']) / df2[f'{s}_unit_iqr']
        df2.drop(columns=['u_med','u_q1','u_q3'], inplace=True)
    return df2

# Flagged dataset
df_flags = add_event_flags(df)
flag_cols = [c for c in df_flags.columns if c.endswith('_drop_flag') or c.endswith('_spike_flag') or c.endswith('_dev_med_iqr') or c.endswith('_dev_unit')]
X2 = df_flags[sensor_cols + flag_cols].copy()
y2 = df_flags['RUL'].astype(float).copy()

# Split again with same strategy
bins2 = df_flags['rul_bin'].astype(str)
sss2 = StratifiedShuffleSplit(n_splits=1, test_size=0.40, random_state=42)
for train_idx, test_idx in sss2.split(X2, bins2):
    X2_train, X2_test = X2.iloc[train_idx], X2.iloc[test_idx]
    y2_train, y2_test = y2.iloc[train_idx], y2.iloc[test_idx]
    bins2_train, bins2_test = bins2.iloc[train_idx], bins2.iloc[test_idx]

sss2_val = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
for tr_idx, val_idx in sss2_val.split(X2_train, bins2_train):
    X2_tr, X2_val = X2_train.iloc[tr_idx], X2_train.iloc[val_idx]
    y2_tr, y2_val = y2_train.iloc[tr_idx], y2_train.iloc[val_idx]

# Train again with flags
metrics2 = []

rf2 = RandomForestRegressor(n_estimators=700, max_depth=16, min_samples_leaf=25, max_features='sqrt', n_jobs=-1, random_state=42)
rf2.fit(X2_tr, y2_tr)
y2_pred_rf = rf2.predict(X2_test)
metrics2.append(evaluate_model('RandomForest+Flags', y2_test, y2_pred_rf))

hgb2 = HistGradientBoostingRegressor(max_depth=6, learning_rate=0.06, max_iter=800, min_samples_leaf=20, l2_regularization=1.0, random_state=42)
hgb2.fit(X2_tr, y2_tr)
y2_pred_hgb = hgb2.predict(X2_test)
metrics2.append(evaluate_model('HistGB+Flags', y2_test, y2_pred_hgb))

lgbm2 = lgb.LGBMRegressor(objective='regression', learning_rate=0.05, num_leaves=31, max_depth=-1, min_child_samples=30, subsample=0.8, colsample_bytree=0.9, reg_lambda=1.0, n_estimators=5000, random_state=42)
lgbm2.fit(X2_tr, y2_tr, eval_set=[(X2_val, y2_val)], eval_metric='rmse', callbacks=[lgb.early_stopping(stopping_rounds=200, verbose=True), lgb.log_evaluation(period=200)])
best_iter2 = getattr(lgbm2, 'best_iteration_', None)
y2_pred_lgbm = lgbm2.predict(X2_test, num_iteration=best_iter2) if best_iter2 else lgbm2.predict(X2_test)
metrics2.append(evaluate_model('LightGBM+Flags', y2_test, y2_pred_lgbm))

cat2 = CatBoostRegressor(loss_function='RMSE', eval_metric='RMSE', depth=6, learning_rate=0.05, l2_leaf_reg=3.0, bootstrap_type='Bernoulli', subsample=0.8, random_strength=0.5, iterations=5000, early_stopping_rounds=200, use_best_model=True, random_seed=42, verbose=200)
cat2.fit(X2_tr, y2_tr, eval_set=(X2_val, y2_val))
y2_pred_cat = cat2.predict(X2_test)
metrics2.append(evaluate_model('CatBoost+Flags', y2_test, y2_pred_cat))

compare_df2 = pd.DataFrame(metrics2).sort_values('RMSE')
print('=== Test Metrics AFTER adding event flags ===')
print(compare_df2)

# Decide & save best
combined = pd.concat([pd.DataFrame(compare_df).assign(Round='Baseline'), pd.DataFrame(compare_df2).assign(Round='WithFlags')], ignore_index=True)
print('=== Combined Comparison (Baseline vs WithFlags) ===')
print(combined.sort_values(['Round','RMSE']))

best_row = combined.sort_values(['RMSE','MAE']).iloc[0]
best_model_name = best_row['Model']
print(f'Selected best model: {best_model_name} ({best_row["Round"]})')

from joblib import dump
if best_model_name == 'RandomForest':
    dump(rf, os.path.join(ART_DIR, 'best_rf.joblib'))
elif best_model_name == 'HistGB':
    dump(hgb, os.path.join(ART_DIR, 'best_hgb.joblib'))
elif best_model_name == 'LightGBM':
    dump(lgbm, os.path.join(ART_DIR, 'best_lgbm.joblib'))
elif best_model_name == 'CatBoost':
    cat.save_model(os.path.join(ART_DIR, 'best_catboost.cbm'))
elif best_model_name == 'RandomForest+Flags':
    dump(rf2, os.path.join(ART_DIR, 'best_rf_flags.joblib'))
elif best_model_name == 'HistGB+Flags':
    dump(hgb2, os.path.join(ART_DIR, 'best_hgb_flags.joblib'))
elif best_model_name == 'LightGBM+Flags':
    dump(lgbm2, os.path.join(ART_DIR, 'best_lgbm_flags.joblib'))
elif best_model_name == 'CatBoost+Flags':
    cat2.save_model(os.path.join(ART_DIR, 'best_catboost_flags.cbm'))

pd.DataFrame(compare_df).to_csv(os.path.join(ART_DIR, 'metrics_baseline.csv'), index=False)
pd.DataFrame(compare_df2).to_csv(os.path.join(ART_DIR, 'metrics_with_flags.csv'), index=False)
combined.to_csv(os.path.join(ART_DIR, 'metrics_combined.csv'), index=False)

print(f'Artifacts saved to: {ART_DIR}')


Drive already mounted!
Loaded preprocessed: (166441, 54)
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.075207 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12506
[LightGBM] [Info] Number of data points in the train set: 79891, number of used features: 50
[LightGBM] [Info] Start training from score 288.777632
Training until validation scores don't improve for 200 rounds
[200]	valid_0's rmse: 26.5286	valid_0's l2: 703.765
[400]	valid_0's rmse: 22.2663	valid_0's l2: 495.788
[600]	valid_0's rmse: 20.1368	valid_0's l2: 405.489
[800]	valid_0's rmse: 18.9196	valid_0's l2: 357.952
[1000]	valid_0's rmse: 18.0669	valid_0's l2: 326.412
[1200]	valid_0's rmse: 17.3689	valid_0's l2: 301.678
[1400]	valid_0's rmse: 16.9294	valid_0's l2: 286.605
[1600]	valid_0's rmse: 16.6099	valid_0's l2: 275.888
[1800]	valid_0's rmse: 16.3357	valid_0's l2: 266.855
[2000]	valid_0's rmse: 16.1402	valid_0's l2: 260.507
[2200]	val